<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!pip install pyLDAvis


import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [28]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [29]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(1)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57


In [30]:
%%time

bug_report = bug_report.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

CPU times: user 114 ms, sys: 2.65 ms, total: 117 ms
Wall time: 125 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [31]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 716 ms, sys: 1.47 ms, total: 717 ms
Wall time: 1.13 s


In [32]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 17.2 s, sys: 19.9 ms, total: 17.2 s
Wall time: 17.3 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [33]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head(1)

CPU times: user 25.2 s, sys: 191 ms, total: 25.4 s
Wall time: 25.4 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [34]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

CPU times: user 10.4 ms, sys: 969 µs, total: 11.4 ms
Wall time: 12.6 ms


,Issue_id,Duplicated_issue,Title,Description
0,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [35]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

CPU times: user 20.1 ms, sys: 0 ns, total: 20.1 ms
Wall time: 19.2 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [36]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 500:
      break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [38]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

CPU times: user 4.09 s, sys: 105 ms, total: 4.2 s
Wall time: 5.48 s


[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1),
 (4543, 2),
 (4544, 1),
 (4545, 1),
 (4546, 1),
 (4547, 1)]

In [39]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. "actually" word seen 1 time.
2. "builders" word seen 2 time.
3. "change" word seen 5 time.
4. "comment" word seen 1 time.
5. "compare" word seen 1 time.
6. "complete" word seen 1 time.
7. "consider" word seen 1 time.
8. "consistency" word seen 1 time.
9. "contain" word seen 1 time.
10. "default" word seen 1 time.
11. "derive" word seen 1 time.
12. "document" word seen 1 time.
13. "drive" word seen 1 time.
14. "editor" word seen 3 time.
15. "effect" word seen 1 time.
16. "ensure" word seen 1 time.
17. "external" word seen 3 time.
18. "extremely" word seen 1 time.
19. "fail" word seen 1 time.
20. "inconsistent" word seen 1 time.
21. "internal" word seen 1 time.
22. "maintain" word seen 2 time.
23. "mandatory" word seen 1 time.
24. "miss" word seen 1 time.
25. "modify" word seen 1 time.
26. "move" word seen 1 time.
27. "navigator" word seen 2 time.
28. "network" word seen 1 time.
29. "occur" word seen 1 time.
30. "open" word seen 1 time.
31. "optimal" word seen 1 time.
32. "problem" wo

In [40]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

CPU times: user 6min 27s, sys: 22.8 s, total: 6min 50s
Wall time: 10min 47s


In [41]:
%%time

for id, topic in lda_model.print_topics():
    print('Topic: {} \nWords: {}'.format(id, topic))

Topic: 0 
Words: 0.025*"action" + 0.022*"extension" + 0.022*"launch" + 0.019*"browser" + 0.017*"command" + 0.013*"support" + 0.013*"define" + 0.011*"context" + 0.011*"property" + 0.011*"content"
Topic: 1 
Words: 0.049*"editor" + 0.030*"eclipse" + 0.025*"window" + 0.024*"select" + 0.022*"dialog" + 0.021*"reproduce" + 0.019*"search" + 0.018*"perspective" + 0.016*"button" + 0.014*"reproducible"
Topic: 2 
Words: 0.076*"thread" + 0.036*"wait" + 0.031*"monitor" + 0.031*"progress" + 0.031*"lock" + 0.023*"method" + 0.022*"memory" + 0.017*"objectwait" + 0.016*"worker" + 0.015*"reference"
Topic: 3 
Words: 0.133*"source" + 0.082*"method" + 0.030*"javathread" + 0.030*"daemon" + 0.027*"thread" + 0.025*"threadblocked" + 0.022*"exception" + 0.016*"available" + 0.014*"cwindowsdll" + 0.014*"native"
Topic: 4 
Words: 0.018*"create" + 0.017*"attachment" + 0.017*"attach" + 0.014*"warn" + 0.013*"scroll" + 0.012*"problem" + 0.012*"control" + 0.011*"console" + 0.011*"widget" + 0.010*"screenshot"
Topic: 5 
Wor

In [43]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.346616306111006

Coherence Score:  0.6362406057081598
CPU times: user 38.2 s, sys: 457 ms, total: 38.6 s
Wall time: 38.3 s


In [44]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

CPU times: user 17.2 s, sys: 288 ms, total: 17.5 s
Wall time: 21.8 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.099747 -0.153337       1        1  15.476808
7     -0.187371 -0.035182       2        1  15.131853
1     -0.189180  0.013884       3        1  14.489917
9     -0.063653 -0.014213       4        1  11.622688
4     -0.148946  0.094422       5        1   9.783017
0     -0.135059 -0.108841       6        1   9.464799
6      0.019476  0.338665       7        1   7.459010
5      0.232526 -0.189643       8        1   6.688395
3      0.275507 -0.029560       9        1   6.471759
2      0.296447  0.083805      10        1   3.411753, topic_info=                  Term          Freq         Total Category  logprob  loglift
32             project  22275.000000  22275.000000  Default  30.0000  30.0000
372             source  15347.000000  15347.000000  Default  29.0000  29.0000
589            message  12643.000000  12643.000000  Default  28.0000  28.0000
116             method  19078.000000  19078.000000  Default  27.0000  27.0000
103            eclipse  29158.000000  29158.000000  Default  26.0000  26.0000
13              editor  12676.000000  12676.000000  Default  25.0000  25.0000
761             import  10439.000000  10439.000000  Default  24.0000  24.0000
360             public   9522.000000   9522.000000  Default  23.0000  23.0000
1341            thread   8037.000000   8037.000000  Default  22.0000  22.0000
52             display  10434.000000  10434.000000  Default  21.0000  21.0000
109             select  14486.000000  14486.000000  Default  20.0000  20.0000
484             bundle   5976.000000   5976.000000  Default  19.0000  19.0000
448             return   7785.000000   7785.000000  Default  18.0000  18.0000
609             window   6966.000000   6966.000000  Default  17.0000  17.0000
101             dialog  10724.000000  10724.000000  Default  16.0000  16.0000
806          exception   6222.000000   6222.000000  Default  15.0000  15.0000
4346       libjvmdylib   3590.000000   3590.000000  Default  14.0000  14.0000
248             update   9081.000000   9081.000000  Default  13.0000  13.0000
466             action   5053.000000   5053.000000  Default  12.0000  12.0000
130            feature   6210.000000   6210.000000  Default  11.0000  11.0000
4365        javathread   3423.000000   3423.000000  Default  10.0000  10.0000
3241            daemon   3393.000000   3393.000000  Default   9.0000   9.0000
626        perspective   4718.000000   4718.000000  Default   8.0000   8.0000
608             search   5452.000000   5452.000000  Default   7.0000   7.0000
1005         extension   3673.000000   3673.000000  Default   6.0000   6.0000
358             plugin   7968.000000   7968.000000  Default   5.0000   5.0000
780             static   3080.000000   3080.000000  Default   4.0000   4.0000
1196            launch   5474.000000   5474.000000  Default   3.0000   3.0000
4374     threadblocked   2879.000000   2879.000000  Default   2.0000   2.0000
698            browser   3206.000000   3206.000000  Default   1.0000   1.0000
349             instal   3051.610325   3052.494016   Topic1  -4.4962   1.8655
239            plugins   2698.201609   2699.085280   Topic1  -4.6193   1.8655
1174          download   2170.499433   2171.383103   Topic1  -4.8369   1.8654
1234          software    857.124181    858.007859   Topic1  -5.7661   1.8648
652           fragment    755.374243    756.257906   Topic1  -5.8924   1.8647
4243           equinox    714.771651    715.655361   Topic1  -5.9477   1.8646
2279      installation    638.496392    639.380043   Topic1  -6.0605   1.8644
1242            releng    516.129412    517.013066   Topic1  -6.2733   1.8641
1955            splash    472.895273    473.778943   Topic1  -6.3608   1.8640
2871          baseline    418.568185    419.451882   Topic1  -6.4828   1.8637
3707       maintenance    298.714941    299.598738   Topic1  -6.8202   1.8629
1930           keyring    287.624

In [ ]:
%%time

for i in range(1):
    exec('topic_{} = pd.DataFrame()'.format(i))
    
    for j in range(len(master_reports)):

        topic = lda_model[dictionary.doc2bow(master_reports.Description[j])]
        topic = np.asarray(topic)

        if int(topic[np.argmax(topic[:,1]),0]) == i:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(i,i))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(i,i))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(i,i))